In [1]:
!pip install mediapipe
import cv2
import mediapipe as mp
import numpy as np
from google.colab.patches import cv2_imshow

mp_pose = mp.solutions.pose

# Setup the Pose function for images - independently for the images standalone processing.
pose_image = mp_pose.Pose(static_image_mode=True, min_detection_confidence=0.8,model_complexity=2)

# Setup the Pose function for videos - for video processing.
pose_video = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.7,
                          min_tracking_confidence=0.7)

# Initialize mediapipe drawing class - to draw the landmarks points.
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

def mpPoseImg(image_pose, pose=pose_image, draw=False, display=False):
    
    annotated_image = image_pose.copy()
    
    image_in_RGB = cv2.cvtColor(image_pose, cv2.COLOR_BGR2RGB)
 
    
    results = pose.process(image_in_RGB)

    if results.pose_landmarks and draw:    
      mp_drawing.draw_landmarks(image=annotated_image,landmark_list=results.pose_landmarks,connections=mp_pose.POSE_CONNECTIONS,
                                landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style())


        # mp_drawing.draw_landmarks(image=original_image, landmark_list=resultant.pose_landmarks,connections=mp_pose.POSE_CONNECTIONS,
        #                           landmark_drawing_spec=mp_drawing.DrawingSpec(color=(255,255,255),thickness=3, circle_radius=3),
        #                           connection_drawing_spec=mp_drawing.DrawingSpec(color=(49,125,237),thickness=2, circle_radius=2))

    if display:
            
            # plt.figure(figsize=[22,22])
            # plt.subplot(121);plt.imshow(image_pose[:,:,::-1]);plt.title("Input Image");plt.axis('off');
            # plt.subplot(122);plt.imshow(annotated_image[:,:,::-1]);plt.title("Pose detected Image");plt.axis('off');
            cv2.imshow(annotated_image)
            return results

    else:
        return annotated_image,results
     


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.6/33.6 MB 22.5 MB/s eta 0:00:00


In [2]:
train_keypoints=[]

numOfKeypoints = 34

import numpy as np
import tensorflow as tf

# calculate the center of the object in each frame

def CentralisedKeypoints1(keypoints):
  frame_centers = []
  for frame_keypoints in keypoints:
    # assume the center is the mean of all keypoints
    center_x = np.nanmean([kp[0] for kp in frame_keypoints if kp[0] is not None])
    center_y = np.nanmean([kp[1] for kp in frame_keypoints if kp[1] is not None])
    frame_centers.append((center_x, center_y))

  relative_keypoints = keypoints

  for j in range(len(relative_keypoints)):               #frame no
    for k in range(len(relative_keypoints[j])):          # each keypoint of frame
      if relative_keypoints[j][k][0] is not None:
          relative_keypoints[j][k][0] -= frame_centers[j][0]
      if relative_keypoints[j][k][1] is not None:
        relative_keypoints[j][k][1] -= frame_centers[j][1]

  
  for i in range(len(relative_keypoints)):   #each frame
    for j in range(len(relative_keypoints[i])):   #34 keypoints
      if relative_keypoints[i][j][0] is None:
        relative_keypoints[i][j][0] = 0.0
      if relative_keypoints[i][j][1] is None:
        relative_keypoints[i][j][1] = 0.0

  return relative_keypoints



In [3]:
numOfKeypoints = 34

import numpy as np
import tensorflow as tf

# calculate the center of the object in each frame

def CentralisedKeypoints2(keypoints):
  frame_centers = []
  for frame_keypoints in keypoints:
    # assume the center is the mean of all keypoints
    center_x = np.nanmean([kp[0] for kp in frame_keypoints if kp[0] is not None])
    center_y = np.nanmean([kp[1] for kp in frame_keypoints if kp[1] is not None])
    frame_centers.append((center_x, center_y))

  relative_keypoints = keypoints

  for j in range(len(relative_keypoints)):               #frame no
    for k in range(len(relative_keypoints[j])):          # each keypoint of frame
      if relative_keypoints[j][k][0] is not None:
          relative_keypoints[j][k][0] -= frame_centers[j][0]
      if relative_keypoints[j][k][1] is not None:
        relative_keypoints[j][k][1] -= frame_centers[j][1]

  
  frames_keypoints=[]
  
  for i in range(len(relative_keypoints)):   #each frame
    flag=1
    for j in range(len(relative_keypoints[i])):   #34 keypoints
      if relative_keypoints[i][j][0] is None or relative_keypoints[i][j][1] is None:
        flag = 0
        break
    
    if(flag == 1) :
      frames_keypoints.append(relative_keypoints[i])
  
  kk = [[0.0,0.0] for _ in range(numOfKeypoints)]
  if(len(frames_keypoints)>0):
    for i in range (16-len(frames_keypoints)):
        frames_keypoints.append(kk)

  return frames_keypoints

In [4]:
numOfKeypoints = 34
def getKeyPointsforFrame(frame):
  img,results = mpPoseImg(frame)
  keypoints = [[None,None] for _ in range(numOfKeypoints)]
  if results.pose_landmarks == None:
    return keypoints
    
  landmarks = results.pose_landmarks
  for i in range(numOfKeypoints-2):
     mark = landmarks.landmark[i]
     keypoints[i][0]=mark.x
     keypoints[i][1]=mark.y
  #------------------------------------------------------------------------------------------------
  try:
    # MID HIP COORDINATES
      left_hip_index = 23
      right_hip_index = 24
      Mid_hip_x =  (keypoints[left_hip_index][0]+keypoints[right_hip_index][0])/2
      Mid_hip_y =(keypoints[left_hip_index][1]+keypoints[right_hip_index][1])/2
      # keypoints.append([Mid_hip_x,Mid_hip_y])
      keypoints[32][0]=Mid_hip_x
      keypoints[32][1]=Mid_hip_y
                          
  except:
      pass
      
  #------------------------------------------------------------------------------------------------
  try:
    # NECK COORDINATES
      left_shoulder_index = 11
      right_shoulder_index = 12
      Neck_x =  (keypoints[left_shoulder_index][0]+keypoints[right_shoulder_index][0])/2
      Neck_y =(keypoints[left_shoulder_index][1]+keypoints[right_shoulder_index][1])/2
      # keypoints.append([Neck_x,Neck_y])
      keypoints[33][0]=Neck_x
      keypoints[33][1]=Neck_y
                          
  except:
      pass

  #------------------------------------------------------------------------------------------------
     
  return keypoints

In [7]:
from google.colab import drive
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras import initializers
from keras.layers import Conv3D, MaxPooling3D,Dense,ZeroPadding3D,Flatten,Activation,BatchNormalization,RandomFlip,RandomRotation,Rescaling
from keras.models import load_model

# This will prompt for authorization.
drive.mount('/MyDrive')

import numpy as np
import pandas as pd
import tensorflow as tf
import os,cv2

import warnings
warnings.filterwarnings("ignore")

data_path = '../MyDrive/MyDrive/Dataset'


labels=[]
for folder in os.listdir(data_path):
    labels.append(folder)
labels.sort() #len = 107

train_videos=[]
train_labels=[]
train_keypoints = []
import traceback
import random

num = 1

for i,folder in enumerate(labels):
  try:
    for video in os.listdir(data_path+'/'+folder):
      print(num)
      num+=1
      video = os.path.join(data_path+'/'+folder+'/'+video)
      # Open the video file
      cap = cv2.VideoCapture(video)

      # Get the total number of frames in the video
      total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

      # Select a random starting frame number
      start_frame = random.randint(0, total_frames - 16)

      # Set the number of frames to be selected
      num_frames = 16

      # Set the frame number to start with
      cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame)

      # Loop through the frames and save them
      frames = []
      keypoints = []
      for j in range(num_frames):
          ret, image = cap.read()
          if ret:
            frames.append(cv2.resize(image,(112,112)))
            kk = getKeyPointsforFrame(image)
            keypoints.append(kk)

      # Release the video file
      cap.release()


      train_videos.append(frames)

      train_keypoints.append(CentralisedKeypoints1(keypoints))
      train_labels.append(i)
      # if(num>2):
      #    break

  except Exception:
    traceback.print_exc()

Mounted at /MyDrive
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69


KeyboardInterrupt: ignored

In [ ]:
import cv2
cap = cv2.VideoCapture(0)


RGBmodel = load_model('RGBStream.h5')
Posemodel = load_model('PoseStream.h5')


video = []

while True:
  ret, frame = cap.read()

  if not ret:
    break
  
  video.append(frame)

  if(len(video)==16):
    keypoints = []
    for image in video:
      keypoints.append(getKeyPointsforFrame(image))

    keypoints = CentralisedKeypoints1(keypoints)

    yogaPose = predictYogaPose(video,keypoints)



In [6]:
# train_videos = np.asarray(train_videos)
train_keypoints = np.asarray(train_keypoints)
train_labels = np.asarray(train_labels).astype('int32')

# train_keypoints = tf.convert_to_tensor(train_keypoints, dtype=tf.float32)
# # yKey_train = tf.cast(yKey_train , dtype=tf.float32)
# train_labels = tf.convert_to_tensor(train_labels, dtype=tf.float32)

# train_keypoints = np.array([np.array(val) for val in train_keypoints])

from sklearn.model_selection import train_test_split

XKey_train,XKey_test,yKey_train,yKey_test = train_test_split(train_keypoints,train_labels,test_size=0.25,shuffle=True)

NameError: ignored

In [ ]:
from keras import models, layers
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, Flatten, LSTM, Dense, Dropout, TimeDistributed, GlobalAveragePooling1D
from keras.optimizers import Adam
from keras.initializers import GlorotNormal
from keras.regularizers import L1L2

yKey_train = np.squeeze(yKey_train)

tf.config.run_functions_eagerly(True)

# Define model input shape
input_shape = (16, 34, 2)

# Build model architecture
model = models.Sequential()

model.add(TimeDistributed(Conv1D(filters=16, kernel_size=3, activation='relu'),input_shape = input_shape))
# model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling1D(pool_size=2)))
model.add(TimeDistributed(Conv1D(filters=128, kernel_size=3, activation='relu')))
# model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling1D(pool_size=2)))
model.add(TimeDistributed(Conv1D(filters=50, kernel_size=3, activation='relu')))
# model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling1D(pool_size=2)))

# flatten output from CNN layers
model.add(TimeDistributed(GlobalAveragePooling1D()))

# add LSTM layer
# model.add(LSTM(100, dropout=0.0, recurrent_dropout=0.0, kernel_initializer=GlorotNormal(seed=42), recurrent_regularizer=L1L2(l1=0.0, l2=0.001), bias_regularizer=L1L2(l1=0.0, l2=0.001), recurrent_initializer=GlorotNormal(seed=42), return_sequences=False))
model.add(LSTM(100, recurrent_regularizer=L1L2(l1=0.01, l2=0.001), bias_regularizer=L1L2(l1=0.01, l2=0.001), recurrent_initializer=GlorotNormal(seed=42), return_sequences=False))

# TODO: forget bias 
# TODO: L1 = 0.01


# add fully connected layers

# model.add(Dropout(0.5))
# model.add(Dense(100, activation='relu'))

model.add(Dropout(0.5))
model.add(Dense(80, activation='relu'))

# add Softmax layer
model.add(Dense(20, activation='softmax'))

# compile model
adam = Adam(learning_rate=0.001) 
model.compile(loss='sparse_categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

# print model summary
# print(model.summary())

history = model.fit(XKey_train,yKey_train,batch_size=32,epochs = 300,validation_split = 0.2)

In [ ]:
softmaxRGB = RGBmodel.predict(X_trainRGB)
softmaxPose = Posemodel.predict(XKey_train)

In [ ]:
import numpy as np

# Combine the softmax outputs by taking the average of their values
combined_softmax = (softmaxRGB*0.7 + softmaxPose*0.3) 

# Generate some example ground truth labels
actualValues = Ytrain

# Convert the combined softmax probabilities to predicted labels
predicted_labels = np.argmax(combined_softmax)

# Compute the accuracy of the predictions
accuracy = np.mean(predicted_labels == actualValues)

# Print the results
print("Combined softmax probabilities:", combined_softmax)
print("Ground truth labels:", actualValues)
print("Predicted labels:", predicted_labels)
print("Accuracy:", accuracy)
